In [127]:
! pip install pickle-mixin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
# 워닝 은닉
import warnings
warnings.filterwarnings(action='ignore')

In [129]:
# 라이브러리 import
import pickle
import pandas as pd
import numpy as np
import datetime
import re

In [130]:
# 시각화 라이브러리 import
import seaborn as sns
import matplotlib.pyplot as plt  
import matplotlib as mpl    
import matplotlib.mlab as mlab
import matplotlib.cm as cm

#그래프 시각화 옵션 설정함수
%matplotlib inline

sns.set(style="ticks", color_codes=True, font_scale=1.2)
color = sns.color_palette()
sns.set_style('darkgrid')

#그래프의 한글을 더욱 선명하게 출력
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

#그래프에서 음수 값이 나올 때, 깨지는 현상 방지
mpl.rc('axes',unicode_minus=False)

In [131]:
# from sklearn import metrics
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
# from scipy import stats
# from scipy.stats import skew, norm, probplot, boxcox
from scipy.stats import norm, probplot
from scipy.spatial.distance import cdist
# import datetime
import math

In [132]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# columns 생략 없이 출력
# pd.set_option('display.max_columns', None)
# 문자열 출력 길이
pd.options.display.max_colwidth = 20

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [134]:
# row 데이터 불러오기
with open('/content/drive/MyDrive/CP2/df_clean2.pickle', 'rb') as f:
    data = pickle.load(f)

In [135]:
df = data.copy()

In [136]:
# 전송 날짜 타입 변경 object -> datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# RFM 생성
recency = df[df["id type"] == "celeb"].groupby(["sender_nickname_secured"]).agg({"created_at":"max"})
frequency = df[df["id type"] == "celeb"].groupby(["sender_nickname_secured"]).agg({"message":"count"})
monetary = df[df["id type"] == "celeb"].groupby(["sender_nickname_secured"]).agg({"message_len":"sum"})

rfm = pd.merge(recency,frequency,how='left',on='sender_nickname_secured')
rfm = pd.merge(rfm,monetary,how='left',on='sender_nickname_secured')
rfm.rename(columns = {"created_at" : "recency", "message" : "frequency", "message_len" : "monetary"}, inplace=True)

# 2022년 09월 17일 23시 59분 39초가 마지막 메세지 전송 시간 : 2022-09-18 00:00:00을 기준으로 설정
current_day = pd.to_datetime('2022-09-18 00:00:00')
# current_day = pd.to_datetime('2022-01-01 00:00:00')
# 최근방문일과 기준 날짜의 시간 차이 (일 기준)
time_diff = (current_day - rfm['recency']) / 86400
# 시간 차이를 초단위로 계산
time_in_seconds = [x.total_seconds() for x in time_diff] 
# 변환된 데이터를 다시 삽입한다.
rfm['recency'] = time_in_seconds

# 인덱스 초기화
rfm.reset_index(drop=False, inplace=True)

In [137]:
# RFM 분포 시각화 함수
def QQ_plot(data, measure):
    fig = plt.figure(figsize=(20,7))

    #Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(data)

    #Kernel Density plot
    fig1 = fig.add_subplot(121)
    sns.distplot(data, fit=norm)
    fig1.set_title(measure + ' Distribution ( mu = {:.2f} and sigma = {:.2f} )'.format(mu, sigma), loc='center')
    fig1.set_xlabel(measure)
    fig1.set_ylabel('Frequency')

    #QQ plot
    fig2 = fig.add_subplot(122)
    res = probplot(data, plot=fig2)
    fig2.set_title(measure + ' Probability Plot (skewness: {:.6f} and kurtosis: {:.6f} )'.format(data.skew(), data.kurt()), loc='center')

    plt.tight_layout()
    plt.show()

## 사분위로 분류

In [158]:
df_segmented = rfm.copy()
describe = df_segmented.describe()
describe

,recency,frequency,monetary
count,303.000000,303.000000,303.000000
mean,18.047138,500.188119,9717.933993
std,32.930259,1144.548314,19771.525810
min,0.000243,1.000000,1.000000
25%,0.234443,9.500000,197.000000
50%,1.967395,114.000000,2329.000000
75%,24.791215,430.500000,9361.000000
max,189.544976,9143.000000,121422.000000


In [159]:
# 받은 메세지
df_segmented['R_score']=pd.cut(rfm["recency"], bins=[0, describe["recency"][4], describe["recency"][5], describe["recency"][6], describe["recency"][7]+1],labels=["4","3","2","1"],right=False)
df_segmented['F_score']=pd.cut(rfm["frequency"], bins=[0, describe["frequency"][4], describe["frequency"][5], describe["frequency"][6], describe["frequency"][7]+1],labels=["1","2","3","4"],right=False)
df_segmented['M_score']=pd.cut(rfm["monetary"], bins=[0, describe["monetary"][4], describe["monetary"][5], describe["monetary"][6], describe["monetary"][7]+1],labels=["1","2","3","4"],right=False)

tp_change = {'R_score':'int', 'F_score':'int', 'M_score':'int'}
df_segmented = df_segmented.astype(tp_change)

df_segmented['RFM_Segment'] = df_segmented.R_score.map(str)+df_segmented.F_score.map(str)+df_segmented.M_score.map(str)
df_segmented['RFM_Score'] = df_segmented[['R_score','F_score','M_score']].sum(axis=1)

In [140]:
"""
df_segmented["Segment"] = "고객"

for i in range(len(df_segmented)):
    if (df_segmented.R_score[i] == 4) & (df_segmented.F_score[i] == 4) & (df_segmented.M_score[i] >= 3):
        df_segmented["Segment"][i] = "1(VIP 고객)"
    elif (3 <= df_segmented.R_score[i] <= 4) & (3 <= df_segmented.F_score[i] <= 4) & (df_segmented.M_score[i] >= 3):
        df_segmented["Segment"][i] = "2(충성 고객)"
    elif (df_segmented.R_score[i] >= 3) & (df_segmented.F_score[i] <= 3) & (1 <= df_segmented.M_score[i] <= 3):
        df_segmented["Segment"][i] = "3(잠재 충성 고객)"
    elif (2 <= df_segmented.R_score[i] <= 3) & (df_segmented.F_score[i] < 3) & (df_segmented.M_score[i] < 3):
        df_segmented["Segment"][i] = "4(잠드려는 고객)"
    elif (df_segmented.R_score[i] < 3) & (2 <= df_segmented.F_score[i] <= 4) & (2 <= df_segmented.M_score[i] <= 4):
        df_segmented["Segment"][i] = "5(이탈 우려 고객)"
    elif (df_segmented.R_score[i] < 2) & (df_segmented.F_score[i] <= 4) & (df_segmented.M_score[i] <= 4):
        df_segmented["Segment"][i] = "6(이탈 고객)"
"""


'\ndf_segmented["Segment"] = "고객"\n\nfor i in range(len(df_segmented)):\n    if (df_segmented.R_score[i] == 4) & (df_segmented.F_score[i] == 4) & (df_segmented.M_score[i] >= 3):\n        df_segmented["Segment"][i] = "1(VIP 고객)"\n    elif (3 <= df_segmented.R_score[i] <= 4) & (3 <= df_segmented.F_score[i] <= 4) & (df_segmented.M_score[i] >= 3):\n        df_segmented["Segment"][i] = "2(충성 고객)"\n    elif (df_segmented.R_score[i] >= 3) & (df_segmented.F_score[i] <= 3) & (1 <= df_segmented.M_score[i] <= 3):\n        df_segmented["Segment"][i] = "3(잠재 충성 고객)"\n    elif (2 <= df_segmented.R_score[i] <= 3) & (df_segmented.F_score[i] < 3) & (df_segmented.M_score[i] < 3):\n        df_segmented["Segment"][i] = "4(잠드려는 고객)"\n    elif (df_segmented.R_score[i] < 3) & (2 <= df_segmented.F_score[i] <= 4) & (2 <= df_segmented.M_score[i] <= 4):\n        df_segmented["Segment"][i] = "5(이탈 우려 고객)"\n    elif (df_segmented.R_score[i] < 2) & (df_segmented.F_score[i] <= 4) & (df_segmented.M_score[i] <= 4):\n 

In [160]:
df_segmented["Segment"] = "고객"

for i in range(len(df_segmented)):
    if (df_segmented.R_score[i] == 4) & (df_segmented.F_score[i] == 4) & (df_segmented.M_score[i] >= 3):
        df_segmented["Segment"][i] = "VIP 등급"
    elif (3 <= df_segmented.R_score[i] <= 4) & (3 <= df_segmented.F_score[i] <= 4) & (3 <= df_segmented.M_score[i] <= 4):
        df_segmented["Segment"][i] = "충성 등급"
    elif (3 <= df_segmented.R_score[i] <= 4) & (1 < df_segmented.F_score[i] <= 3) & (1 < df_segmented.M_score[i] <= 3):
        df_segmented["Segment"][i] = "적극적 잠재 등급"
    elif (2 <= df_segmented.R_score[i] <= 4) & (df_segmented.F_score[i] < 3) & (df_segmented.M_score[i] < 3):
        df_segmented["Segment"][i] = "소극적 잠재 등급"
    elif (1 < df_segmented.R_score[i] < 3) & (2 <= df_segmented.F_score[i] <= 4) & (2 <= df_segmented.M_score[i] <= 4):
        df_segmented["Segment"][i] = "관심 필요 등급"
    elif (df_segmented.R_score[i] < 2) & (df_segmented.F_score[i] <= 4) & (df_segmented.M_score[i] <= 4):
        df_segmented["Segment"][i] = "이탈 등급"

In [161]:
# 등급별 평균
df_segmented.groupby("Segment").mean()

,recency,frequency,monetary,R_score,F_score,M_score,RFM_Score
Segment,,,,,,,
VIP 등급,0.107269,1871.333333,34182.500000,4.000000,4.000000,3.976190,11.976190
관심 필요 등급,5.584014,494.794118,10729.058824,2.000000,3.323529,3.176471,8.500000
소극적 잠재 등급,7.328919,23.480000,466.460000,2.240000,1.480000,1.500000,5.220000
이탈 등급,63.825995,43.000000,858.565789,1.000000,1.460526,1.447368,3.907895
적극적 잠재 등급,0.614732,57.617647,1459.294118,3.352941,2.058824,2.176471,7.588235
충성 등급,0.534307,742.343284,15013.298507,3.268657,3.313433,3.343284,9.925373


In [162]:
# 등급별 인원
df_segmented.groupby("Segment").count()

,sender_nickname_secured,recency,frequency,monetary,R_score,F_score,M_score,RFM_Segment,RFM_Score
Segment,,,,,,,,,
VIP 등급,42,42,42,42,42,42,42,42,42
관심 필요 등급,34,34,34,34,34,34,34,34,34
소극적 잠재 등급,50,50,50,50,50,50,50,50,50
이탈 등급,76,76,76,76,76,76,76,76,76
적극적 잠재 등급,34,34,34,34,34,34,34,34,34
충성 등급,67,67,67,67,67,67,67,67,67


In [163]:
# 등급 구분 완료 csv 저장
df_segmented.to_csv('/content/drive/MyDrive/CP2/df_segmented.csv', index=False, encoding="utf-8-sig")

In [150]:
# 각각의 닉네임 기준 주차별 등급 나누기
df_resample = df.set_index('created_at',drop=False)

nick_list = df_resample[df_resample["id type"] == "celeb"]["sender_nickname_secured"].unique()

for i, nick in enumerate(nick_list):
    # globals()[str(i)] = df_resample[df_resample["sender_nickname_secured"] == nick]
    globals()['df_'+str(i)] = pd.DataFrame()
    globals()['df_'+str(i)]['recency'] = df_resample[df_resample["sender_nickname_secured"] == nick].created_at.resample('1W').max()
    globals()['df_'+str(i)]['frequency'] = df_resample[df_resample["sender_nickname_secured"] == nick].message.resample('1W').count().cumsum()
    globals()['df_'+str(i)]['monetary'] = df_resample[df_resample["sender_nickname_secured"] == nick].message_len.resample('1W').sum().cumsum()
    globals()['df_'+str(i)]["created_at"] = pd.to_datetime(globals()['df_'+str(i)].index)
    # 기준날짜
    current_day = globals()['df_'+str(i)]["created_at"] + datetime.timedelta(days=1)
    # 최근방문일과 기준 날짜의 시간 차이 (일 기준)
    time_diff = (current_day - globals()['df_'+str(i)]['recency']) / 86400
    # 시간 차이를 초단위로 계산
    time_in_seconds = [x.total_seconds() for x in time_diff] 
    # 변환된 데이터를 다시 삽입한다.
    globals()['df_'+str(i)]['recency'] = time_in_seconds

    globals()['df_'+str(i)].dropna(inplace=True)

    recency = globals()['df_'+str(i)]["recency"]
    frequency = globals()['df_'+str(i)]["frequency"]
    monetary = globals()['df_'+str(i)]["monetary"]

    globals()['df_'+str(i)]["nickname"] = nick

    globals()['df_'+str(i)]['R_score']=pd.cut(globals()['df_'+str(i)]["recency"], bins=[0, describe["recency"][4], describe["recency"][5], describe["recency"][6], describe["recency"][7]+1],labels=["4","3","2","1"],right=False)
    globals()['df_'+str(i)]['F_score']=pd.cut(globals()['df_'+str(i)]["frequency"], bins=[0, describe["frequency"][4], describe["frequency"][5], describe["frequency"][6], describe["frequency"][7]+1],labels=["1","2","3","4"],right=False)
    globals()['df_'+str(i)]['M_score']=pd.cut(globals()['df_'+str(i)]["monetary"], bins=[0, describe["monetary"][4], describe["monetary"][5], describe["monetary"][6], describe["monetary"][7]+1],labels=["1","2","3","4"],right=False)

    tp_change = {'R_score':'int', 'F_score':'int', 'M_score':'int'}
    globals()['df_'+str(i)] = globals()['df_'+str(i)].astype(tp_change)

    globals()['df_'+str(i)]['RFM_Segment'] = globals()['df_'+str(i)].R_score.map(str)+globals()['df_'+str(i)].F_score.map(str)+globals()['df_'+str(i)].M_score.map(str)
    globals()['df_'+str(i)]['RFM_Score'] = globals()['df_'+str(i)][['R_score','F_score','M_score']].sum(axis=1)

    # globals()['df_'+str(i)]['R_score']=pd.cut(globals()['df_'+str(i)]["recency"], bins=[0, recency.quantile(.25), recency.quantile(.50), recency.quantile(.75), recency.quantile(1)+1],labels=["4","3","2","1"],right=False)
    # globals()['df_'+str(i)]['F_score']=pd.cut(globals()['df_'+str(i)]["frequency"], bins=[0, frequency.quantile(.25), frequency.quantile(.50), frequency.quantile(.75), frequency.quantile(1)+1],labels=["1","2","3","4"],right=False)
    # globals()['df_'+str(i)]['M_score']=pd.cut(globals()['df_'+str(i)]["monetary"], bins=[0, monetary.quantile(.25), monetary.quantile(.50), monetary.quantile(.75), monetary.quantile(1)+1],labels=["1","2","3","4"],right=False)

    globals()['df_'+str(i)]["Segment"] = "고객"

    for j in range(len(globals()['df_'+str(i)])):
        if (globals()['df_'+str(i)].R_score[j] == 4) & (globals()['df_'+str(i)].F_score[j] == 4) & (globals()['df_'+str(i)].M_score[j] >= 3):
            globals()['df_'+str(i)]["Segment"][j] = "VIP 고객"
        elif (3 <= globals()['df_'+str(i)].R_score[j] <= 4) & (3 <= globals()['df_'+str(i)].F_score[j] <= 4) & (globals()['df_'+str(i)].M_score[j] >= 3):
            globals()['df_'+str(i)]["Segment"][j] = "충성 고객"
        elif (globals()['df_'+str(i)].R_score[j] >= 3) & (globals()['df_'+str(i)].F_score[j] <= 3) & (1 <= globals()['df_'+str(i)].M_score[j] <= 3):
            globals()['df_'+str(i)]["Segment"][j] = "잠재 충성 고객"
        elif (2 <= globals()['df_'+str(i)].R_score[j] <= 3) & (globals()['df_'+str(i)].F_score[j] < 3) & (globals()['df_'+str(i)].M_score[j] < 3):
            globals()['df_'+str(i)]["Segment"][j] = "잠드려는 고객"
        elif (globals()['df_'+str(i)].R_score[j] < 3) & (2 <= globals()['df_'+str(i)].F_score[j] <= 4) & (2 <= globals()['df_'+str(i)].M_score[j] <= 4):
            globals()['df_'+str(i)]["Segment"][j] = "이탈 우려 고객"
        elif (globals()['df_'+str(i)].R_score[j] < 2) & (globals()['df_'+str(i)].F_score[j] <= 4) & (globals()['df_'+str(i)].M_score[j] <= 4):
            globals()['df_'+str(i)]["Segment"][j] = "이탈 고객"

In [151]:
df_123

,recency,frequency,monetary,created_at,nickname,R_score,F_score,M_score,RFM_Segment,RFM_Score,Segment
created_at,,,,,,,,,,,
2022-08-07,0.236076,28,592,2022-08-07,도롱*,3,2,2,322,7,잠재 충성 고객
2022-08-14,0.167847,100,2021,2022-08-14,도롱*,4,2,2,422,8,잠재 충성 고객
2022-08-21,1.133668,121,2570,2022-08-21,도롱*,3,3,3,333,9,충성 고객
2022-08-28,0.151134,162,3464,2022-08-28,도롱*,4,3,3,433,10,충성 고객
2022-09-04,1.049224,175,3863,2022-09-04,도롱*,3,3,3,333,9,충성 고객
2022-09-11,3.358564,186,4042,2022-09-11,도롱*,2,3,3,233,8,이탈 우려 고객
2022-09-18,1.031087,297,6487,2022-09-18,도롱*,3,3,3,333,9,충성 고객


In [152]:
df_127

,recency,frequency,monetary,created_at,nickname,R_score,F_score,M_score,RFM_Segment,RFM_Score,Segment
created_at,,,,,,,,,,,
2022-07-31,0.484432,54,564,2022-07-31,이걸*,3,2,2,322,7,잠재 충성 고객
2022-08-07,5.319328,58,594,2022-08-07,이걸*,2,2,2,222,6,잠드려는 고객
2022-08-14,6.616041,59,606,2022-08-14,이걸*,2,2,2,222,6,잠드려는 고객


In [153]:
df_concat = pd.DataFrame()
for i in range(len(nick_list)):
    df_concat = pd.concat([df_concat, globals()['df_'+str(i)]])

In [154]:
# 등급별 평균
df_concat.groupby("Segment").mean()

,recency,frequency,monetary,R_score,F_score,M_score,RFM_Score
Segment,,,,,,,
VIP 고객,0.098560,1601.455556,33399.159259,4.000000,4.000000,3.929630,11.929630
이탈 우려 고객,3.440700,438.241667,9441.295833,2.000000,3.233333,3.220833,8.454167
잠드려는 고객,3.846997,25.852140,541.287938,2.000000,1.591440,1.591440,5.182879
잠재 충성 고객,0.668347,56.045576,1261.155496,3.327078,1.914209,1.951743,7.193029
충성 고객,0.676730,705.654174,14377.330494,3.235094,3.393526,3.396934,10.025554


In [155]:
# 등급별 인원
df_concat.groupby("Segment").count()

,recency,frequency,monetary,created_at,nickname,R_score,F_score,M_score,RFM_Segment,RFM_Score
Segment,,,,,,,,,,
VIP 고객,270,270,270,270,270,270,270,270,270,270
이탈 우려 고객,240,240,240,240,240,240,240,240,240,240
잠드려는 고객,257,257,257,257,257,257,257,257,257,257
잠재 충성 고객,373,373,373,373,373,373,373,373,373,373
충성 고객,587,587,587,587,587,587,587,587,587,587


In [156]:
# 등급 구분 완료 csv 저장
df_concat.to_csv('/content/drive/MyDrive/CP2/df_segmented_w.csv', index=False, encoding="utf-8-sig")